# **Obtaining Dataset**

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option("display.max_columns",50)
pd.set_option("display.max_rows",50)

import nltk
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud   #graphical representation of data
from wordcloud import STOPWORDS   #insignificant words without any meaning

from PIL import Image

from sklearn.model_selection import train_test_split


from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords

stopwords = stopwords.words('english') + ['of','the','to','be','that','with','on','for','by','at','one']



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [3]:
df_fake=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
df_true=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

In [4]:
print("Fake Data shape: ",df_fake.shape)
print("True Data shape: ",df_true.shape)

Fake Data shape:  (23481, 4)
True Data shape:  (21417, 4)


In [5]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
df_fake.insert(1, 'Fake/True', 'Fake')
df_true.insert(1, 'Fake/True', 'True')
df=df_true.append(df_fake)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      44898 non-null  object
 1   Fake/True  44898 non-null  object
 2   text       44898 non-null  object
 3   subject    44898 non-null  object
 4   date       44898 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


In [8]:
df['Fake/True'].value_counts()

Fake    23481
True    21417
Name: Fake/True, dtype: int64

In [9]:
df['subject'].value_counts()

politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

In [10]:
df=df.drop(columns=['title','date'],axis=1)

# **Preprocessing the Data**

In [11]:
df['label']=df['Fake/True'].map({"Fake":0,"True":1})

In [12]:
df['text']=df['text'].str.lower()
df['text']=df['text'].str.replace('[^\w\s]','')
df['text']=df['text'].str.replace('\d+','')
df['text']=df['text'].str.replace('\n','').replace('\r',"")

In [13]:
del df['subject']

In [14]:
x=df['text'].values
y=df['label'].values

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #Both CountVectorizer and TfidfVectorizer are used for text feature extraction, specifically for converting text documents into numerical feature vectors that can be used as input for machine learning algorithms.

In [15]:
vect=CountVectorizer(stop_words='english')
x=vect.fit_transform(x)

In [16]:
x_train,x_test,y_train,y_test  = train_test_split(x,y,random_state=69,test_size=.15)

# **Modeling with ML algorithms**

**Naive Bayes**

In [17]:
nb = MultinomialNB()
nb.fit(x_train,y_train)
prediction_nb = nb.predict(x_test)


**Decision Tree**

In [18]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
prediction_dt = dt.predict(x_test)

**Random Forest**

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
prediction_rf = rf.predict(x_test)

**Neural Network**

In [20]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(max_iter=300)
nn.fit(x_train,y_train)
prediction_nn = nn.predict(x_test)

In [22]:
print("Naive Bayes Accuracy : \n")
accuracy_score(y_test,prediction_nb)

Naive Bayes Accuracy : 



0.955902004454343

In [23]:
print("Decision Tree Accuracy : \n")
accuracy_score(y_test,prediction_dt)

Decision Tree Accuracy : 



0.9971789161098737

In [24]:
print("Random Forest Accuracy : \n")
accuracy_score(y_test,prediction_rf)

Random Forest Accuracy : 



0.9910913140311804

In [25]:
print("Neural Network Accuracy : \n")
accuracy_score(y_test,prediction_nn)

Neural Network Accuracy : 



0.9949517446176689

# **Perform evaluation and Comparative analysis**

**Neural Network**

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,prediction_nn))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3550
           1       0.99      1.00      0.99      3185

    accuracy                           0.99      6735
   macro avg       0.99      1.00      0.99      6735
weighted avg       0.99      0.99      0.99      6735



**Naive Bayes**

In [32]:
print(classification_report(y_test,prediction_nb))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96      3550
           1       0.94      0.97      0.95      3185

    accuracy                           0.96      6735
   macro avg       0.96      0.96      0.96      6735
weighted avg       0.96      0.96      0.96      6735



**Decision Tree**

In [33]:
print(classification_report(y_test,prediction_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3550
           1       1.00      1.00      1.00      3185

    accuracy                           1.00      6735
   macro avg       1.00      1.00      1.00      6735
weighted avg       1.00      1.00      1.00      6735



**Random Forest**

In [34]:
print(classification_report(y_test,prediction_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3550
           1       0.99      0.99      0.99      3185

    accuracy                           0.99      6735
   macro avg       0.99      0.99      0.99      6735
weighted avg       0.99      0.99      0.99      6735

